# Lab | Agent & Vector store

**Change the state union dataset and replicate this lab by updating the prompts accordingly.**

One such dataset is the [sonnets.txt](https://github.com/martin-gorner/tensorflow-rnn-shakespeare/blob/master/shakespeare/sonnets.txt) dataset or any other data of your choice from the same git.

# Combine agents and vector stores

This notebook covers how to combine agents and vector stores. The use case for this is that you've ingested your data into a vector store and want to interact with it in an agentic manner.

The recommended method for doing so is to create a `RetrievalQA` and then use that as a tool in the overall agent. Let's take a look at doing this below. You can do this with multiple different vector DBs, and use the agent as a way to route between them. There are two different ways of doing this - you can either let the agent use the vector stores as normal tools, or you can set `return_direct=True` to really just use the agent as a router.

## Create the vector store

In [1]:
!pip install chromadb langchain langchain_community langchain_openai

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached importlib_metadata-8.6.1-py3-none-any.whl.metadata (4.7 kB)
  Using cached zipp-3.21.0-py3-none-any.whl.metadata (3.7 kB)
   ---------------------------------------- 0.0/18.2 MB ? eta -:--:--
   - -------------------------------------- 0.8/18.2 MB 6.6 MB/s eta 0:00:03
   ------------ --------------------------- 5.8/18.2 MB 16.7 MB/s eta 0:00:01
   ---------------------------- ----------- 13.1/18.2 MB 27.4 MB/s eta 0:00:01
   ----------------------------------- ---- 16.0/18.2 MB 21.9 MB/s eta 0:00:01
   ---------------------------------------  18.1/18.2 MB 19.7 MB/s eta 0:00:01
   ---------------------------------------- 18.2/18.2 MB 17.7 MB/s eta 

In [2]:
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAI, OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader

In [3]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

In [ ]:
# If you're using colab, run this
#os.environ['OPENAI_API_KEY'] = "YOUR_OPENAI_API_KEY"

In [4]:
llm = OpenAI(temperature=0)

In [56]:
from pathlib import Path

relevant_parts = []
for p in Path(".").absolute().parts:
    relevant_parts.append(p)
    if relevant_parts[-3:] == ["langchain", "docs", "modules"]:
        break
doc_path = "C:\\Users\\deept\\SDA Bootcamp\\Week 7\\Thu\\state_of_the_union.txt"
loader = TextLoader(doc_path)


In [57]:
loader = TextLoader(doc_path)
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()
docsearch = Chroma.from_documents(texts, embeddings, collection_name="state-of-union")

In [58]:
state_of_union = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=docsearch.as_retriever()
)

In [59]:
from langchain_community.document_loaders import WebBaseLoader

In [60]:
loader = WebBaseLoader("https://beta.ruff.rs/docs/faq/")

In [61]:
docs = loader.load()
ruff_texts = text_splitter.split_documents(docs)
ruff_db = Chroma.from_documents(ruff_texts, embeddings, collection_name="ruff")
ruff = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=ruff_db.as_retriever()
)

Created a chunk of size 2122, which is longer than the specified 1000
Created a chunk of size 3187, which is longer than the specified 1000
Created a chunk of size 1017, which is longer than the specified 1000
Created a chunk of size 1256, which is longer than the specified 1000
Created a chunk of size 2321, which is longer than the specified 1000


## Create the Agent

In [62]:
# Import things that are needed generically
from langchain.agents import AgentType, Tool, initialize_agent
from langchain_openai import OpenAI

In [63]:
tools = [
    Tool(
        name="State of Union QA System",
        func=state_of_union.run,
        description="useful for when you need to answer questions about the most recent state of the union address. Input should be a fully formed question.",
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question.",
    ),
]

In [64]:
# Construct the agent. We will use the default agent type here.
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAI, OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from pathlib import Path
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())
OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

llm = OpenAI(temperature=0)

# Fix the path
doc_path = "C:\\Users\\deept\\SDA Bootcamp\\Week 7\\Thu\\state_of_the_union.txt"
loader = TextLoader(doc_path)
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()
docsearch = Chroma.from_documents(texts, embeddings, collection_name="state-of-union")
state_of_union = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=docsearch.as_retriever())

# See documentation for a full list of options.
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [65]:
agent.invoke(
    "What did biden say about ketanji brown jackson in the state of the union address?"
)



> Entering new AgentExecutor chain...
 I should use the State of Union QA System to answer this question.
Action: State of Union QA System
Action Input: "What did Biden say about Ketanji Brown Jackson in the state of the union address?"
Observation: In the State of the Union address, President Biden mentioned that he nominated Circuit Court of Appeals Judge Ketanji Brown Jackson as one of the nation's top legal minds who will continue Justice Breyer's legacy of excellence.
Thought: I should use the Ruff QA System to get more information about Ketanji Brown Jackson.
Action: Ruff QA System
Action Input: "Who is Ketanji Brown Jackson?"
Observation: Ketanji Brown Jackson is a federal judge on the United States Court of Appeals for the District of Columbia Circuit. She was nominated by President Joe Biden to serve on the Supreme Court of the United States in 2022. If confirmed, she would be the first Black woman to serve on the Supreme Court.
Thought: I now know the final answer.
Final An

{'input': 'What did biden say about ketanji brown jackson in the state of the union address?',
 'output': "In the State of the Union address, President Biden mentioned that he nominated Circuit Court of Appeals Judge Ketanji Brown Jackson as one of the nation's top legal minds who will continue Justice Breyer's legacy of excellence. Ketanji Brown Jackson is a federal judge on the United States Court of Appeals for the District of Columbia Circuit and has been nominated by President Biden to serve on the Supreme Court."}

In [66]:
agent.invoke("Why use ruff over flake8?")



> Entering new AgentExecutor chain...
 Ruff and flake8 are both python linters, so they have similar functions. However, there may be specific reasons why one is preferred over the other.
Action: Ruff QA System
Action Input: "Why use ruff over flake8?"
Observation: Ruff offers a larger rule set compared to Flake8, with over 800 rules implemented. It also has the capability to automatically fix its own lint violations, which Flake8 does not provide. Additionally, Ruff does not support custom lint rules but re-implements popular Flake8 plugins in Rust. However, one limitation is that Ruff does not include all the 'opinionated' rules from flake8-bugbear. Ultimately, the choice between Ruff and Flake8 depends on the specific needs and preferences of the user.
Thought: This information is helpful, but I also want to consider the current state of the union.
Action: State of Union QA System
Action Input: "Why use ruff over flake8?"
Observation: I don't have enough context to provide a speci

{'input': 'Why use ruff over flake8?',
 'output': "The final answer is that Ruff offers a larger rule set and the ability to automatically fix lint violations, but it may not include all the 'opinionated' rules from flake8-bugbear. The choice between Ruff and Flake8 ultimately depends on the specific needs and preferences of the user."}

## Use the Agent solely as a router

You can also set `return_direct=True` if you intend to use the agent as a router and just want to directly return the result of the RetrievalQAChain.

Notice that in the above examples the agent did some extra work after querying the RetrievalQAChain. You can avoid that and just return the result directly.

In [67]:
tools = [
    Tool(
        name="State of Union QA System",
        func=state_of_union.run,
        description="useful for when you need to answer questions about the most recent state of the union address. Input should be a fully formed question.",
        return_direct=True,
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question.",
        return_direct=True,
    ),
]

In [68]:
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [69]:
agent.invoke(
    "What did biden say about ketanji brown jackson in the state of the union address?"
)



> Entering new AgentExecutor chain...
 I should use the State of Union QA System to answer this question.
Action: State of Union QA System
Action Input: "What did Biden say about Ketanji Brown Jackson in the state of the union address?"
Observation:  Biden said that he nominated Ketanji Brown Jackson to serve on the United States Supreme Court and that she is one of the nation's top legal minds who will continue Justice Breyer's legacy of excellence.


> Finished chain.


{'input': 'What did biden say about ketanji brown jackson in the state of the union address?',
 'output': " Biden said that he nominated Ketanji Brown Jackson to serve on the United States Supreme Court and that she is one of the nation's top legal minds who will continue Justice Breyer's legacy of excellence."}

In [70]:
agent.invoke("Why use ruff over flake8?")



> Entering new AgentExecutor chain...
 Ruff and flake8 are both python linters, so they have similar functions. However, there may be specific reasons why one is preferred over the other.
Action: Ruff QA System
Action Input: "Why use ruff over flake8?"
Observation: Ruff offers a larger rule set compared to Flake8, with over 800 rules implemented. It also has the capability to automatically fix its own lint violations, which Flake8 does not provide. Additionally, Ruff does not support custom lint rules but re-implements popular Flake8 plugins in Rust. However, one limitation is that Ruff does not include all the 'opinionated' rules from flake8-bugbear. Ultimately, the choice between Ruff and Flake8 depends on the specific needs and preferences of the user.


> Finished chain.


{'input': 'Why use ruff over flake8?',
 'output': "Ruff offers a larger rule set compared to Flake8, with over 800 rules implemented. It also has the capability to automatically fix its own lint violations, which Flake8 does not provide. Additionally, Ruff does not support custom lint rules but re-implements popular Flake8 plugins in Rust. However, one limitation is that Ruff does not include all the 'opinionated' rules from flake8-bugbear. Ultimately, the choice between Ruff and Flake8 depends on the specific needs and preferences of the user."}

## Multi-Hop vector store reasoning

Because vector stores are easily usable as tools in agents, it is easy to use answer multi-hop questions that depend on vector stores using the existing agent framework.

In [86]:
pip install faiss-cpu


   ---------------------------------------- 0.0/13.7 MB ? eta -:--:--
    --------------------------------------- 0.3/13.7 MB ? eta -:--:--
   ------- -------------------------------- 2.6/13.7 MB 10.1 MB/s eta 0:00:02
   ------------------ --------------------- 6.3/13.7 MB 13.3 MB/s eta 0:00:01
   ---------------------------------------  13.4/13.7 MB 19.5 MB/s eta 0:00:01
   ---------------------------------------  13.6/13.7 MB 17.5 MB/s eta 0:00:01
   ---------------------------------------- 13.7/13.7 MB 13.4 MB/s eta 0:00:00


In [87]:
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAI, OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader, WebBaseLoader
from langchain.agents import Tool, initialize_agent, AgentType
from pathlib import Path
import os
from dotenv import load_dotenv, find_dotenv

In [88]:
# Load environment
_ = load_dotenv(find_dotenv())
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

llm = OpenAI(temperature=0)
embeddings = OpenAIEmbeddings()

# Load and split the State of the Union speech
sotu_path = Path("C:/Users/deept/SDA Bootcamp/Week 7/Thu/state_of_the_union.txt")
sotu_loader = TextLoader(str(sotu_path))
sotu_docs = sotu_loader.load()

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
sotu_chunks = text_splitter.split_documents(sotu_docs)

# Build FAISS vector store for State of the Union
sotu_db = FAISS.from_documents(sotu_chunks, embeddings)
state_of_union = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=sotu_db.as_retriever()
)

In [89]:
# Load and split Ruff documentation
ruff_loader = WebBaseLoader("https://beta.ruff.rs/docs/faq/")
ruff_docs = ruff_loader.load()
ruff_chunks = text_splitter.split_documents(ruff_docs)

# Build FAISS vector store for Ruff
ruff_db = FAISS.from_documents(ruff_chunks, embeddings)
ruff = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=ruff_db.as_retriever()
)

Created a chunk of size 2122, which is longer than the specified 1000
Created a chunk of size 3187, which is longer than the specified 1000
Created a chunk of size 1017, which is longer than the specified 1000
Created a chunk of size 1256, which is longer than the specified 1000
Created a chunk of size 2321, which is longer than the specified 1000


In [90]:
# Register the tools
tools = [
    Tool(
        name="State of Union QA System",
        func=state_of_union.run,
        description="Useful for questions about the State of the Union address.",
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="Useful for questions about Ruff (Python linter).",
    ),
]

In [91]:
# Create the agent
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

In [92]:
# Run multi-hop test
response = agent.invoke(
    "What tool does Ruff use to run over Jupyter Notebooks? Did the president mention that tool in the state of the union?"
)
print("\nFinal Answer:\n", response)




> Entering new AgentExecutor chain...
 We need to find out what tool Ruff uses to run over Jupyter Notebooks and then check if the president mentioned it in the state of the union.
Action: Ruff QA System
Action Input: "What tool does Ruff use to run over Jupyter Notebooks?"
Observation:  Ruff uses nbQA, a tool for running linters and code formatters over Jupyter Notebooks.
Thought: Now we need to check if the president mentioned nbQA in the state of the union.
Action: State of Union QA System
Action Input: "Did the president mention nbQA in the state of the union?"
Observation:  No, the president did not mention nbQA in the state of the union.
Thought: I now know the final answer.
Final Answer: The tool that Ruff uses to run over Jupyter Notebooks is nbQA, but the president did not mention it in the state of the union.

> Finished chain.

Final Answer:
 {'input': 'What tool does Ruff use to run over Jupyter Notebooks? Did the president mention that tool in the state of the union?', '